In [1]:
import bqplot
from bqplot import LinearScale, Lines, Figure, Axis, DateScale, DateColorScale, Scatter, Tooltip
from ipywidgets import Layout, Label
from bqplot.interacts import BrushIntervalSelector
import ipywidgets as widgets
import numpy as np
import pandas as pd
import qgrid

periods = 1000
security_1 = np.cumsum(np.random.randn(periods)) + 100.
security_2 = security_1 + np.cumsum(np.random.randn(periods))*0.5 
dates = pd.date_range(start='06-01-2007', periods=periods, freq='d')
df = pd.DataFrame(index=dates, data={'security_1':security_1, 'security_2':security_2})
qg = qgrid.show_grid(df)

In [2]:
def plot_and_brush_df(df):
    
    margins = dict(left=25, top=50, bottom=50, right=25)

    x_dt = DateScale()
    scale_y = LinearScale()
    line = Lines(x=dates, y=security_1, scales={'x': x_dt, 'y': scale_y})
    ax_x = Axis(scale=x_dt, grid_lines='solid')
    ax_y = Axis(scale=scale_y, orientation='vertical', tick_format='0.0f')
    brush = BrushIntervalSelector(scale=x_dt)
    fig = Figure(axes=[ax_x, ax_y], marks=[line], fig_margin=margins, interaction=brush)


    # we will share the same y scale
    x_dt2 = DateScale()
    ax_x2 = Axis(scale=x_dt2, grid_lines='solid')
    ax_y2 = Axis(scale=scale_y, orientation='vertical',tick_format='0.0f')
    line2 = Lines(x=[], y=[], scales={'x': x_dt2, 'y': scale_y})
    fig2 = Figure(axes=[ax_x2, ax_y2], marks=[line2], fig_margin=margins)

    def event_handler(name):
        sel = brush.selected
        if sel is None or len(sel) < 2:
            line2.x=[]
            line2.y=[]
        else:
            t_0 = pd.to_datetime(str(sel[0]))
            t_1 = pd.to_datetime(str(sel[1]))
            df_selected = df[t_0:t_1]
            line2.x=df_selected.index
            line2.y=df_selected.values.T[0]

    brush.observe(event_handler)
    return fig, fig2

In [3]:
f1, f2 = plot_and_brush_df(df)

In [4]:
widgets.HBox([f1, f2])

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [6]:
def plot_and_brush_scatter(df):

    lr = LinearRegression()
    
    margins = dict(left=25, top=50, bottom=50, right=25)

    x_dt = DateScale()
    scale_y = LinearScale()
    line_c = Lines(x=[], y=[], scales={'x': x_dt, 'y': scale_y})
    line_c.opacities = [0.65, 0.90]
    ax_x = Axis(scale=x_dt, grid_lines='solid')
    ax_y = Axis(scale=scale_y, orientation='vertical', tick_format='0.0f')
    brush_s = BrushIntervalSelector(scale=x_dt)
    fig_comp = Figure(axes=[ax_x, ax_y], marks=[line_c], fig_margin=margins, interaction=brush_s)

    color_scale = DateColorScale(scheme='Blues')
    scale_s_x = LinearScale()
    scale_s_y = LinearScale()
    reg_line = Lines(x=[], y=[], scales={'x': scale_s_x, 'y': scale_s_y})

    label = Label()
    def on_hover(w, target):
        label.value = dots.dates[target['data']['index']].strftime("%b-%Y")

    dots = Scatter(x=[], y=[], color=[], scales={'x': scale_s_x, 'y': scale_s_y, 'color': color_scale},
                  tooltip=label, unhovered_style={'opacity':0.40})
    dots.on_hover(on_hover)

    ax_s_x = Axis(scale=scale_s_x, grid_lines='solid')
    ax_s_y = Axis(scale=scale_s_x, orientation='vertical')
    
    fig_s = Figure(axes=[ax_s_x, ax_s_y], marks=[dots, reg_line], fig_margin=margins)

        
    def event_handler(name):
        global dates
        sel = brush_s.selected
        if sel is None or len(sel) < 2:
            dots.x = []
            dots.y = []
            reg_line.x = []
            reg_line.y = []
            fig_s.title = ''
        else:
            t_0 = pd.to_datetime(str(sel[0]))
            t_1 = pd.to_datetime(str(sel[1]))
            df = pd.DataFrame(data={'a':fig_comp.marks[0].y[0], 'b':fig_comp.marks[0].y[1]}, index=fig_comp.marks[0].x)
            df_selected = df[t_0:t_1]
            dots.x = df_selected.values[:,0]
            dots.y = df_selected.values[:,1]
            dots.dates = df_selected.index   # you can stick anything into dots (?)
            #dots.color = df_selected.index
            fig_s.title = fig_comp.title
            try:
                lr.fit(dots.x[:, np.newaxis], dots.y)
                # clear them first so the line animation isn't janky
                y = lr.predict(dots.x[:, np.newaxis])
                r2 = r2_score(dots.y, y)
                reg_line.opacities = [max(r2, 0.15)]
                reg_line.x = []
                reg_line.y = []            
                reg_line.x = dots.x
                reg_line.y = y
            except:
                reg_line.x = []
                reg_line.y = []

    brush_s.observe(event_handler)
    
    fig_comp.marks[0].y = [df.values[:,0], df.values[:,1]] # joint.values[:,1] #
    fig_comp.marks[0].x = df.index.values
    
    return fig_comp, fig_s, brush_s

In [7]:
f3, f4, brush = plot_and_brush_scatter(df)

In [8]:
widgets.HBox([f3, f4])